<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/Model_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
# python libraties
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
import scipy.stats as stats
# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [4]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

# ***Some Setup***

The cell below creates the pandas dataframes for training and testing.

In [5]:
#loading dataframes # เอาที่อยู่ของไฟล์เชื่อมกับที่อยู่ของไฟล์เรา โดยกำหนดชื่่อไฟล์ที่เราต้องการ
train = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/train.csv")
validation = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/validation.csv")
test = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/boneage-test-dataset.csv")

In [6]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [7]:
train # แสดง data 'train'

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [8]:
validation # แสดง data 'validation'

,id,boneage,male
0,10186,10,True
1,10758,15,False
2,10860,180,False
3,10877,180,False
4,11008,180,False
...,...,...,...
2517,15605,50,False
2518,15606,113,False
2519,15608,55,False
2520,15609,150,True


In [9]:
test # แสดง data 'test'

,Case ID,Sex,boneage
0,4360,M,169
1,4361,M,170
2,4362,M,73
3,4363,M,153
4,4364,M,135
...,...,...,...
195,4555,F,133
196,4556,F,129
197,4557,F,168
198,4558,F,136


In [10]:
#appending file extension to id column for both training and testing dataframes
# ต่อท้าย data ใน column 'id' ด้วย .png
train['id'] = train['id'].apply(lambda x: str(x)+'.png') # ทำการเปลี่ยน type ใน column 'id' จาก int  เป็น string เพื่อให้สามารถต่อท้ายด้วย .png ได้ # ฟังก์ชันแลมบ์ดาเป็นฟังก์ชันที่ไม่ระบุตัวตนขนาดเล็ก
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png')
test['Case ID'] = test['Case ID'].apply(lambda x: str(x)+'.png')

train.head() # แสดงค่าของ trainบางส่วน

,id,boneage,male
0,1377.png,180,False
1,1378.png,12,False
2,1379.png,94,False
3,1380.png,120,True
4,1381.png,82,False


In [11]:
train['gender'] = train['male'].apply(lambda x: '0' if x else '1') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
train['gender']

0        1
1        1
2        1
3        0
4        1
        ..
10084    1
10085    1
10086    1
10087    1
10088    1
Name: gender, Length: 10089, dtype: object

In [12]:
validation['gender'] = validation['male'].apply(lambda x: '0' if x else '1') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
validation['gender']

0       0
1       1
2       1
3       1
4       1
       ..
2517    1
2518    1
2519    1
2520    0
2521    0
Name: gender, Length: 2522, dtype: object

In [13]:
train['gender'] = train['gender'].apply(lambda x: str(x)) 
validation['gender'] = validation['gender'].apply(lambda x: str(x))

In [14]:
print(train['gender'].value_counts()) #แสดงจำนวนเพศหญิงและเพศชายทั้งหมดใน trian_df

0    5468
1    4621
Name: gender, dtype: int64


In [15]:
train.shape # ข้อมูลมีทั้งหมด 261 จากทั้งหมด 4 column

(10089, 4)

In [16]:
class ImageDataset(Dataset):

    def __init__(self, pickle_file, image_path):
        self.image_dir = image_path
        self.pickle_file = pickle_file
        self.labels = pd.read_csv(pickle_file)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        labels = self.labels.iloc[idx, 0:]
        images = Image.open(f"{self.image_dir}{labels['id']}.png")
        images = np.array(images)
        #image = image[..., :3]

        images = transforms.functional.to_tensor(images)
        images = transforms.functional.resize(images, size=[500,500])
        return images

In [17]:
image_datatrain = ImageDataset(path+"/train.csv", image_path=f"{train_img}")
image_datatrain

In [18]:
image_dataval = ImageDataset(path+"/validation.csv", image_path=f"{val_img}")
image_dataval

In [19]:
image_datatest = ImageDataset(path+"/boneage-test-dataset.csv", image_path=f"{test_img}")
image_datatest

In [20]:
strat_epoch =0 
num_epochs=60
lr=0.0001
patience=2 
batch_size=20
num_workers=0

In [21]:
train_dataloader = DataLoader(image_datatrain, batch_size,num_workers)
train_dataloader

In [22]:
val_dataloader = DataLoader(image_dataval, batch_size,num_workers)
val_dataloader

In [23]:
test_dataloader = DataLoader(image_datatest, batch_size,num_workers)
test_dataloader

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()

        # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.max_pool1(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out
net = Net()
print(net)

Net(
  (conv_layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (relu1): ReLU()
)


In [25]:
optimizer = torch.optim.SGD(net.parameters(), lr)
loss_func = torch.nn.MSELoss()

In [26]:
model = Net()
criterion = nn.L1Loss().to(device)

In [27]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [28]:
total_loss_train, total_acc_train = [],[]
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0)
        # print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [29]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [30]:
best_val_acc = 0
total_loss_val, total_acc_val = [],[]
for epoch in range(1, num_epochs+1):
    loss_train, acc_train = train(train_dataloader, model, criterion, optimizer, epoch)
    loss_val, acc_val = validate(val_dataloader, model, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

ValueError: ignored